# Esercizio di manipolazione dei dati

L'80% del lavoro di un data scientist consiste nella cosiddetta "pulizia dei dati" o *data wrangling* (mentre nel restante 20% del tempo solitamente ci si lamenta di dover pulire i dati). Nelle prossime due lezioni lavoreremo su un caso più pratico.

Nella cartella `data` sono presenti dei file `.csv` (*comma separated values*) che rappresentano delle tabelle disordinate. L'esercizio richiede di:

1. Leggere tutti i CSV.
2. Concatenarli in un unico dataset.
3. Ordinarli.
4. Eliminare le righe con valori doppi.
5. Cambiare il nome di una o più colonne.
5. Modificare dei valori direttamente.

E poi disegnare qualche grafico.

Cominciamo importando `pandas`.

In [1]:
import pandas as pd

Per vedere i contenuti di una cartella (in inglese, più propriamente si dice *directory*), possiamo usare il `!` per usare un comando del terminale: `ls` (abbreviazione di *list*):

In [2]:
!ls -1 ./data

2022_05_24-17_23_33-checkpoint-labels_annotations.csv
2022_05_24-17_53_48-checkpoint-labels_annotations.csv
2022_05_24-17_54_49-checkpoint-labels_annotations.csv
2022_05_24-19_37_44-checkpoint-labels_annotations.csv
2022_05_24-20_09_25-checkpoint-labels_annotations.csv
2022_05_31-17_36_01-checkpoint-labels_annotations.csv
2022_05_31-20_32_30-checkpoint-labels_annotations.csv
2022_05_31-20_44_48-checkpoint-labels_annotations.csv
2022_05_31-22_39_35-checkpoint-labels_annotations.csv
checkpoints.zip


Ma possiamo anche usare un modulo built-in di Python, che si chiama `os` (*operative system*, cioè per manipolare i contenuti del vostro computer):

In [4]:
import os
os.getcwd()

'/Users/luca/Documents/dev/python-projects/lectures/statistica-big-data-economico-aziendali/lezione_10'

In [5]:
os.listdir()

['.DS_Store',
 'esercizio-manipolazione_dati.ipynb',
 '.ipynb_checkpoints',
 'data']

# 1. Leggere i `.csv`

## 1.1 Leggere un file di prova

Leggiamo uno dei CSV per capire che colonne ha:

In [6]:
test_checkpoint = pd.read_csv("./data/2022_05_24-17_23_33-checkpoint-labels_annotations.csv")
test_checkpoint.head()

,roof_id,imageURL,photos_folder,annotation,annotation_time
0,1,2e69e5ad-53e9-daca-b4c4-b9ed9505858b.png,small_photos-api_upload,0,2022_05_24-17_15_18
1,266,c6944d60-8f49-d368-75f3-b151d77d2e3e.png,small_photos-api_upload,1,2022_05_24-17_17_06
2,370,02c68e36-1016-35e2-a9a9-6072e54f2e39.png,small_photos-api_upload,0,2022_05_24-17_20_24
3,421,75da75a6-5bf9-e070-527c-6a59b6f929ec.png,small_photos-api_upload,0,2022_05_24-17_21_51
4,925,ffcda7c4-cfd3-c84a-facd-a385a3afde28.png,small_photos-api_upload,1,2022_05_24-17_17_58


Si noti che il *path* (o percorso) al file è `./data/<nome_del_file>`; come abbiamo detto più volte:

- `.` indica la cartella in cui ci troviamo ora (che possiamo vedere con `os.getcwd()` o con `!pwd`)
- `/` è il path separator (`\` su Windows)
- `data` è la cartella con i dati

Vediamo tutti i CSV in una volta:

In [7]:
os.listdir("./data")

['2022_05_31-20_32_30-checkpoint-labels_annotations.csv',
 '.DS_Store',
 '2022_05_24-17_23_33-checkpoint-labels_annotations.csv',
 '2022_05_24-19_37_44-checkpoint-labels_annotations.csv',
 '2022_05_31-17_36_01-checkpoint-labels_annotations.csv',
 '2022_05_24-17_54_49-checkpoint-labels_annotations.csv',
 '2022_05_24-20_09_25-checkpoint-labels_annotations.csv',
 '2022_05_24-17_53_48-checkpoint-labels_annotations.csv',
 '.ipynb_checkpoints',
 'checkpoints.zip',
 '2022_05_31-20_44_48-checkpoint-labels_annotations.csv',
 '2022_05_31-22_39_35-checkpoint-labels_annotations.csv']

Nella cartella ci sono dei file che non vanno bene: `.DS_Store` e `.ipynb_checkpoints`. Per rimuoverli abbiamo diverse scelte:

### 1.1.1 Molto poco originale

Il più banale, usando sostanzialmente tutte le cose viste finora:

In [8]:
lista_di_file = os.listdir("./data")

file_da_rimuovere = [".DS_Store", ".ipynb_checkpoints"]

for file in file_da_rimuovere:
    lista_di_file.remove(file)
    
lista_di_file

['2022_05_31-20_32_30-checkpoint-labels_annotations.csv',
 '2022_05_24-17_23_33-checkpoint-labels_annotations.csv',
 '2022_05_24-19_37_44-checkpoint-labels_annotations.csv',
 '2022_05_31-17_36_01-checkpoint-labels_annotations.csv',
 '2022_05_24-17_54_49-checkpoint-labels_annotations.csv',
 '2022_05_24-20_09_25-checkpoint-labels_annotations.csv',
 '2022_05_24-17_53_48-checkpoint-labels_annotations.csv',
 'checkpoints.zip',
 '2022_05_31-20_44_48-checkpoint-labels_annotations.csv',
 '2022_05_31-22_39_35-checkpoint-labels_annotations.csv']

Perché non "va bene"? Perché non si tratta di un approccio **scalabile**, che cioè funziona quando il numero di file che abbiamo è molto grande. Immaginate che ci siano un numero imprecisato di file da escludere: non possiamo mica stare ad annotarli a mano in una lista uno per uno. Ovviamente, quindi, esisterà un modo persino più rapido per risolvere il problema.

### 1.1.2 Un nuovo metodo per le stringhe

Alla fine vogliamo solo tenere tutti i file che finiscono per `.csv`. In questi casi, bisogna sempre pensare che il nostro problema è abbastanza comune e che qualcuno lo avrà già risolto: per cui solitamente si fa una ricerca su Google - o meglio su StackOverflow direttamente.

In realtà, è ancora meglio cercare se una cosa così non esiste già in Python: il metodo `string.startswith()` o `string.endswith()` è quello che cerchiamo.

In [9]:
lista_di_file = []

files = os.listdir("./data")

for file in files:
    if file.endswith("checkpoint-labels_annotations.csv"):
        lista_di_file.append(file)
    
lista_di_file

['2022_05_31-20_32_30-checkpoint-labels_annotations.csv',
 '2022_05_24-17_23_33-checkpoint-labels_annotations.csv',
 '2022_05_24-19_37_44-checkpoint-labels_annotations.csv',
 '2022_05_31-17_36_01-checkpoint-labels_annotations.csv',
 '2022_05_24-17_54_49-checkpoint-labels_annotations.csv',
 '2022_05_24-20_09_25-checkpoint-labels_annotations.csv',
 '2022_05_24-17_53_48-checkpoint-labels_annotations.csv',
 '2022_05_31-20_44_48-checkpoint-labels_annotations.csv',
 '2022_05_31-22_39_35-checkpoint-labels_annotations.csv']

### 1.1.3 Approfondimento: List comprehensions

Le list comprehensions sono una sintassi elegante per scrivere il codice sopra ma senza for loop e liste vuote. Dei tutorial molto brevi e ben fatti sono qui:

- [calmcode.io](https://calmcode.io/comprehensions/introduction.html) ha dei video brevissimi e fatti benissimo
- [realpython.com](https://realpython.com/list-comprehension-python/) la solita fonte dettagliata
- [programiz.com](https://www.programiz.com/python-programming/list-comprehension) stessa cosa ma spiegata da qualcun altro

Vediamo un esempio pratico. La struttura di una comprehension è sostanzialmente la stessa di un for loop, ma "appiattito" su una riga:

`a = [x for x in range(1000)]`

`x for x in range(1000)` è letteralmente la stessa sintassi del for loop, solo che l'oggetto (`x`) è davanti. Questo perché possiamo eseguire qualsiasi operazione che vogliamo con `x`:

`a = [x**2 for x in range(1000) if x % 2 == 0]`

La comprehension sopra fa questo: costruisce una lista elevando al quadrato ogni `x` nel range da 0 a 1000, se `x` è pari.

In [10]:
a = [x**2 for x in range(1000) if x % 2 == 0]
print(a[:30])

[0, 4, 16, 36, 64, 100, 144, 196, 256, 324, 400, 484, 576, 676, 784, 900, 1024, 1156, 1296, 1444, 1600, 1764, 1936, 2116, 2304, 2500, 2704, 2916, 3136, 3364]


Diventa molto facile estendere questa logica a operazioni più "concrete":

In [11]:
# list comprehensions
lista_di_file = [
    file                                                     # semplicemente il nome del file
    for file in os.listdir("./data")                         # per ogni file nella nostra directory `data`
    if file.endswith("-checkpoint-labels_annotations.csv")   # solo se il file finisce per queste parole
]

lista_di_file

['2022_05_31-20_32_30-checkpoint-labels_annotations.csv',
 '2022_05_24-17_23_33-checkpoint-labels_annotations.csv',
 '2022_05_24-19_37_44-checkpoint-labels_annotations.csv',
 '2022_05_31-17_36_01-checkpoint-labels_annotations.csv',
 '2022_05_24-17_54_49-checkpoint-labels_annotations.csv',
 '2022_05_24-20_09_25-checkpoint-labels_annotations.csv',
 '2022_05_24-17_53_48-checkpoint-labels_annotations.csv',
 '2022_05_31-20_44_48-checkpoint-labels_annotations.csv',
 '2022_05_31-22_39_35-checkpoint-labels_annotations.csv']

## 1.2 Concatenare i CSV
Costruiamo la lista di DataFrame

In [12]:
lista_di_checkpoint = []

for file in lista_di_file:
    full_path = f"./data/{file}"
    
    print(f"leggendo {full_path = }")
    
    lista_di_checkpoint.append(pd.read_csv(full_path))

leggendo full_path = './data/2022_05_31-20_32_30-checkpoint-labels_annotations.csv'
leggendo full_path = './data/2022_05_24-17_23_33-checkpoint-labels_annotations.csv'
leggendo full_path = './data/2022_05_24-19_37_44-checkpoint-labels_annotations.csv'
leggendo full_path = './data/2022_05_31-17_36_01-checkpoint-labels_annotations.csv'
leggendo full_path = './data/2022_05_24-17_54_49-checkpoint-labels_annotations.csv'
leggendo full_path = './data/2022_05_24-20_09_25-checkpoint-labels_annotations.csv'
leggendo full_path = './data/2022_05_24-17_53_48-checkpoint-labels_annotations.csv'
leggendo full_path = './data/2022_05_31-20_44_48-checkpoint-labels_annotations.csv'
leggendo full_path = './data/2022_05_31-22_39_35-checkpoint-labels_annotations.csv'


A questo punto basta usare una funzione di pandas per "metterli uno dietro l'altro":

In [13]:
dataset_with_duplicates = pd.concat(lista_di_checkpoint)

### 1.2.1 Approfondimento: list comprehensions, parte 2

L'intera operazione è persino più rapida usando le comprehensions, come abbiamo visto sopra:

In [15]:
dataset_with_duplicates = pd.concat([
    pd.read_csv(f"./data/{file}")
    for file in os.listdir("data")
    if file.endswith("-checkpoint-labels_annotations.csv")
])

Se la comprehension è lunga, spezzatela su più righe: ogni `for` o `if` è una nuova riga.

Si veda come possiamo applicare a ogni `file` persino la funzione `pd.read_csv` (attenzione a usare il path giusto!): la lista che otterremo sarà esattamente uguale a quella che abbiamo ottenuto sopra. Una volta acquisita dimestichezza con Python, usare le comprehensions sarà più naturale, perché è più simile al linguaggio parlato.

# 2.Manipolazione dei dati

A questo punto abbiamo un dataset ma con diversi doppioni:

In [16]:
dataset_with_duplicates.head()

,roof_id,imageURL,photos_folder,annotation,annotation_time
0,2,696d9474-17da-f9ff-c854-4a4ee9707153.png,small_photos,0,2022_05_31-20_24_35
1,65,b8126876-4511-f5a3-fc2e-df219213fce3.png,small_photos,1,2022_05_31-20_28_21
2,91,b592f575-c1d9-1e4c-4157-c9af24479231.png,small_photos,0,2022_05_31-19_56_33
3,133,9bfcb9b7-e606-e5a7-ad20-0d77f3c2e0e0.png,small_photos,1,2022_05_31-20_09_18
4,250,8a185d91-7b32-ba97-9dac-5146c6093ca0.png,small_photos,0,2022_05_31-20_06_18


Per vedere quanti, usiamo la funzione `sort_values`, che ha come argomento la lista di colonne che vogliamo mettere in ordine:

In [17]:
dataset_with_duplicates.sort_values(["roof_id", "annotation_time"]).head(10)

,roof_id,imageURL,photos_folder,annotation,annotation_time
0,1,2e69e5ad-53e9-daca-b4c4-b9ed9505858b.png,small_photos-api_upload,0,2022_05_24-17_15_18
0,1,2e69e5ad-53e9-daca-b4c4-b9ed9505858b.png,small_photos-api_upload,0,2022_05_24-17_15_18
0,1,2e69e5ad-53e9-daca-b4c4-b9ed9505858b.png,small_photos-api_upload,0,2022_05_24-17_15_18
0,1,2e69e5ad-53e9-daca-b4c4-b9ed9505858b.png,small_photos-api_upload,0,2022_05_24-17_15_18
0,2,696d9474-17da-f9ff-c854-4a4ee9707153.png,small_photos,0,2022_05_31-20_24_35
0,2,696d9474-17da-f9ff-c854-4a4ee9707153.png,small_photos,0,2022_05_31-20_24_35
0,2,696d9474-17da-f9ff-c854-4a4ee9707153.png,small_photos,0,2022_05_31-20_24_35
1,3,696d9474-17da-f9ff-c854-4a4ee9707153.png,small_photos-api_upload,0,2022_05_24-17_55_35
1,3,696d9474-17da-f9ff-c854-4a4ee9707153.png,small_photos-api_upload,0,2022_05_24-17_55_35
2,12,a4f311db-4697-f6fc-1157-f4880638ad53.png,small_photos-api_upload,1,2022_05_24-18_00_04


Per eliminare i duplicati basta una ricerca su Google: "how to drop duplicates in pandas". Tra i primissimi risultati ci sarà [questa pagina](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html) della documentazione ufficiale.

In [18]:
(
    dataset_with_duplicates
    .sort_values(["roof_id", "annotation_time"])
    .drop_duplicates(["roof_id", "annotation_time"])
    .head()
)

,roof_id,imageURL,photos_folder,annotation,annotation_time
0,1,2e69e5ad-53e9-daca-b4c4-b9ed9505858b.png,small_photos-api_upload,0,2022_05_24-17_15_18
0,2,696d9474-17da-f9ff-c854-4a4ee9707153.png,small_photos,0,2022_05_31-20_24_35
1,3,696d9474-17da-f9ff-c854-4a4ee9707153.png,small_photos-api_upload,0,2022_05_24-17_55_35
2,12,a4f311db-4697-f6fc-1157-f4880638ad53.png,small_photos-api_upload,1,2022_05_24-18_00_04
3,26,baa577f4-4518-9aad-9a25-dfd2b95afabb.png,small_photos-api_upload,0,2022_05_24-18_01_44


Quando dovete concatenare più comandi di pandas è bene usare le parentesi tonde per racchiudere l'espressione e separare ogni metodo su una riga diversa.

# Riassunto

Più semplicemente, quindi, il nostro problema si può risolvere cosi:

In [19]:
data = (
    pd.concat([
        pd.read_csv(f"./data/{file}")
        for file in os.listdir("./data")
        if file.endswith("checkpoint-labels_annotations.csv")
    ])
    .sort_values(["roof_id", "annotation_time"])
    .drop_duplicates(["roof_id", "annotation_time"])
)

data.head()

,roof_id,imageURL,photos_folder,annotation,annotation_time
0,1,2e69e5ad-53e9-daca-b4c4-b9ed9505858b.png,small_photos-api_upload,0,2022_05_24-17_15_18
0,2,696d9474-17da-f9ff-c854-4a4ee9707153.png,small_photos,0,2022_05_31-20_24_35
1,3,696d9474-17da-f9ff-c854-4a4ee9707153.png,small_photos-api_upload,0,2022_05_24-17_55_35
2,12,a4f311db-4697-f6fc-1157-f4880638ad53.png,small_photos-api_upload,1,2022_05_24-18_00_04
3,26,baa577f4-4518-9aad-9a25-dfd2b95afabb.png,small_photos-api_upload,0,2022_05_24-18_01_44


# Ripasso: Group by

Molto frequentemente dobbiamo calcolare delle misure aggregate. Lo abbiamo visto nel dataset dei pinguini, quando abbiamo cercato di calcolare la media della lunghezza del becco in base al sesso. Queste operazioni sono chiamate *group by*, perché il dataset viene "raggruppato" per i vari livelli di una variabile (nel caso dei pinguini, il sesso) e viene calcolata una *funzione di aggregazione* (in questo caso la media) su una o più variabili.

Possiamo anche farlo nel nostro esercizio, calcolando ad esempio quante volte ogni `roof_id` compare nel dataset con i doppioni. Come per ogni cosa in Python, ci sono molti modi per fare le groupby:

## 1. Velocemente con poche colonne

In [27]:
(
    dataset_with_duplicates
    .groupby(["roof_id"])                       # colonna da usare per la groupby
    [["roof_id"]]                               # colonna su cui calcolare la funzione
    .count()                                    # funzione che applichiamo a ciascun livello della variabile
    .rename(columns={"roof_id": "count"})       # rinominiamo la nuova colonna "roof_id" in "count"
    .sort_values(by="count", ascending=False)   # ordiniamo i valori in ordine decrescente
    .head(10)
)

,count
roof_id,
421,7
6018,6
7118,6
266389,6
9387,6
6237,6
56656,6
5418,6
10369,6


Notate come diventa essenziale per scrivere del codice leggibile usare le parentesi e separare ciascun metodo su più righe.

## 2. Con più colonne e più elegante

Torniamo ai nostri pinguini:

In [20]:
import seaborn as sns
import numpy as np

penguins = sns.load_dataset("penguins")

penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


Possiamo raggruppare per più variabili:

In [22]:
(
    penguins
    .groupby(["species", "sex"])          # raggruppiamo per specie e sesso
    [["bill_length_mm"]]                  # prendiamo la colonna bill_length
    .mean()                               # e calcoliamone la media
)

bill_length_mm
species   sex                   
Adelie    Female       37.257534
          Male         40.390411
Chinstrap Female       46.573529
          Male         51.094118
Gentoo    Female       45.563793
          Male         49.473770

In questo caso, potremmo passare direttamente una stringa e non una lista di colonne:

In [29]:
(
    penguins
    .groupby(["species", "sex"])
    ["bill_length_mm"]                 # ! un solo gruppo di []
    .mean()
)

species    sex   
Adelie     Female    37.257534
           Male      40.390411
Chinstrap  Female    46.573529
           Male      51.094118
Gentoo     Female    45.563793
           Male      49.473770
Name: bill_length_mm, dtype: float64

Ma come vedete l'output non è molto elegante: in generale solo i DataFrame vengono resi con una tabella "elegante". Se specificate una sola colonna con una stringa, otterrete in automatico una Series (cioè un dataset di una sola colonna), mentre se passate una lista (anche di un solo elemento) otterrete un DataFrame.

Ma possiamo addirittura calcolare la stessa funzione su più colonne:

In [28]:
(
    penguins
    .groupby(["species", "sex"])
    [["bill_length_mm", "bill_depth_mm"]]
    .mean()
)

bill_length_mm  bill_depth_mm
species   sex                                  
Adelie    Female       37.257534      17.621918
          Male         40.390411      19.072603
Chinstrap Female       46.573529      17.588235
          Male         51.094118      19.252941
Gentoo    Female       45.563793      14.237931
          Male         49.473770      15.718033

Se non indichiamo nessuna colonna su cui operare, la statistica verrà calcolata su tutte le colonne su cui può essere calcolata.

In [34]:
(
    penguins
    .groupby(["species", "sex"])
    .mean()
)

bill_length_mm  bill_depth_mm  flipper_length_mm  \
species   sex                                                        
Adelie    Female       37.257534      17.621918         187.794521   
          Male         40.390411      19.072603         192.410959   
Chinstrap Female       46.573529      17.588235         191.735294   
          Male         51.094118      19.252941         199.911765   
Gentoo    Female       45.563793      14.237931         212.706897   
          Male         49.473770      15.718033         221.540984   

                  body_mass_g  
species   sex                  
Adelie    Female  3368.835616  
          Male    4043.493151  
Chinstrap Female  3527.205882  
          Male    3938.970588  
Gentoo    Female  4679.741379  
          Male    5484.836066

Possiamo anche indicare più funzioni per colonne diverse usando `.agg` o `.aggregate`:

In [35]:
(
    penguins
    .groupby(["species", "sex"])
    .agg({
        "bill_length_mm": ["mean", "std"],    # le funzioni più "note" si possono passare come stringhe
        "bill_depth_mm": [np.mean, np.std],   # ma `.agg` accetta qualsiasi funzione, anche scritta da noi
        "body_mass_g": "max"
    })
)

bill_length_mm           bill_depth_mm           body_mass_g
                           mean       std          mean       std         max
species   sex                                                                
Adelie    Female      37.257534  2.028883     17.621918  0.942993      3900.0
          Male        40.390411  2.277131     19.072603  1.018886      4775.0
Chinstrap Female      46.573529  3.108669     17.588235  0.781128      4150.0
          Male        51.094118  1.564558     19.252941  0.761273      4800.0
Gentoo    Female      45.563793  2.051247     14.237931  0.540249      5200.0
          Male        49.473770  2.720594     15.718033  0.741060      6300.0